In [1]:
import requests
import pandas as pd
import time
import talib
import pytz
import datetime
import os
from ib_insync import *

In [2]:
util.startLoop()
ib = IB()

In [3]:
ticker = str(input('Enter ticker: ')).upper() 
count = int(input('Amount: ')) 

Enter ticker: spy
Amount: 10


In [4]:
if not ib.isConnected():  # connect only without active connection    
    try:
        print('Connecting...')
        ib.connect('127.0.0.1', 7497, clientId=1)  # connect
    except Exception as ex:
        print('Error:', ex)  # catch at exception

if ib.isConnected():
    print("Successfully connected")

def balance():
    balances = {av.tag: float(av.value) for av in ib.accountSummary()
                if av.tag in ['AvailableFunds', 'BuyingPower', 'TotalCashValue', 'NetLiquidation']}
    balance = balances.get('AvailableFunds', 0)

    return balance

def check_balance(ticker, count):
    price = read_data(ticker)['close'].iloc[-1]
    amount = price * count
    if balance() > amount:
        return True
    else:
        return False

def time_check(ticker):
    contract = Stock(ticker)
    cds = ib.reqContractDetails(contract)
    hours = cds[0].tradingHours
    hourslist1 = hours.split(';')
    hourslist2 = hourslist1[0].split('-')
    hourslistopening = hourslist2[0].split(':')
    tz = get_tz(ticker)
    today = datetime.datetime.now(tz=pytz.UTC).astimezone(pytz.timezone(tz))
    date = today.strftime("%Y%m%d")
    time = today.strftime("%H%M")
    print('DateTime: ', today.strftime("%d-%m-%Y at %I:%M%p"), tz)
    
    if hourslistopening[1] == 'CLOSED':
        return 'close' 
    else:
        hourslistclosing = hourslist2[1].split(':')
        openingsdict = dict(zip(hourslistopening[::2], hourslistopening[1::2]))
        closingdict = dict(zip(hourslistclosing[::2], hourslistclosing[1::2]))
        hoursregular = cds[0].liquidHours
        hourslist1regular = hoursregular.split(';')
        hourslist2regular = hourslist1regular[0].split('-')
        hourslistopeningregular = hourslist2regular[0].split(':')
        hourslistclosingregular = hourslist2regular[1].split(':')
        openingsdictregular = dict(zip(hourslistopeningregular[::2], hourslistopeningregular[1::2]))
        closingdictregular = dict(zip(hourslistclosingregular[::2], hourslistclosingregular[1::2]))

        rangelist = []

        for key, value in openingsdict.items():
            rangelist.append(value)
    
        for key, value in closingdict.items():
            rangelist.append(value)

        for key, value in openingsdictregular.items():
            rangelist.append(value)
    
        for key, value in closingdictregular.items():
            rangelist.append(value)    
       
        sortrangelist = sorted(rangelist) 

        if sortrangelist[0] <= time < sortrangelist[1]:
            return 'premarket' 
        elif sortrangelist[1] <= time < sortrangelist[2]:
            return 'regular' 
        elif sortrangelist[2] <= time < sortrangelist[3]:
            return 'postmarket'
        else:
            return 'close'
                 
def load_data(ticker):
    filename = f'{ticker}.csv'
    interval = '1min'
    apikey = 'RZPE1GE7KZJ66X9N'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&apikey={apikey}&datatype=csv'
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(filename, "w") as file:
        file.write(content)
        
    print(f'\tUpdate {ticker} price history:', filename)
    
def get_tz(ticker):
    filename = f'{ticker}.json'
    interval = '1min'
    apikey = 'RZPE1GE7KZJ66X9N'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&apikey={apikey}'
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(filename, "w") as file:
        file.write(content)
    data = pd.read_json(f'{ticker}.json')
    tz = data['Meta Data']['6. Time Zone']   
    os.remove(f'{ticker}.json')
    
    return tz
    
def read_data(ticker):
    data = pd.read_csv(f'{ticker}.csv')
    os.remove(f'{ticker}.csv')
        
    return data

def algorithm(ticker):
    try:
        close = read_data(ticker)['close'].fillna(method='ffill')
    except KeyError:
        print('No data, the limit of requests may be exceeded')
        disconnect()
    else:
        ma_long = talib.SMA(close, timeperiod=26)
        ma_short = talib.SMA(close, timeperiod=9)

        last_price = close.iloc[-1]

        print(f'\tCheck SMA for {ticker} (last={last_price:.2f}): Short={ma_short.iloc[-1]:.4f} Long={ma_long.iloc[-1]:.4f}')
        print(f'\tCheck SMA for {ticker} (last={last_price:.2f}): Short={ma_short.iloc[-2]:.4f} Long={ma_long.iloc[-2]:.4f} (previous)')

        if ma_long.iloc[-2] > ma_short.iloc[-2] and ma_long.iloc[-1] < ma_short.iloc[-1]:
                contract = get_contract(ticker)
                buy(ticker, contract)

        elif ma_long.iloc[-2] < ma_short.iloc[-2] and ma_long.iloc[-1] > ma_short.iloc[-1]:
                contract = get_contract(ticker)
                sell(ticker, contract)
        
def get_contract(ticker):
    contract = Stock(f'{ticker}', 'SMART', 'USD')
    ib.qualifyContracts(contract)
    
    return contract
    
def list_positions():
    positions = ib.positions()
    positions = "\n".join([f"{p.contract.localSymbol} {p.position}x{p.avgCost}"
                               for p in positions])
    return positions

def list_orders():
    trades = ib.openTrades()
    orders = "\n".join([f"{t.order.action} {t.contract.secType} {t.contract.symbol} {t.contract.localSymbol}"
                            f" {t.order.totalQuantity}x{t.order.lmtPrice}"
                            for t in trades])
    return orders

def buy(ticker, contract): 
    if f'{ticker}' not in list_positions():
        if f'{ticker}' not in list_orders():
            order = MarketOrder('BUY', f'{count}') 
            ib.placeOrder(contract, order)
        
def sell(ticker, contract):
    if f'{ticker}' in list_positions():
         if f'{ticker}' not in list_orders():
            order = MarketOrder('SELL',  f'{count}')
            ib.placeOrder(contract, order)
            
def run_algorithm():
    messages = {
      'premarket': 'Premarket',
      'postmarket': 'Postmarket',
      'close': 'Closed',
      'regular': 'Regular session',
    } 
    message = messages.get(time_check(ticker), 'Uncertain')
    print(message)
        
    if message == 'Regular session':
        load_data(ticker) 
        if check_balance(ticker, count):        
            print('== Start working ==')

            while True:

                load_data(ticker)
                algorithm(ticker)
                time.sleep(60)
                
        else:
            print('Insufficient funds')
    else:
        print('Wait for the regular session to open')

def disconnect():
    if ib.isConnected():  # check for connection
        ib.disconnect()  # disconnect

        while ib.isConnected():  # wait while disconnecting
            time.sleep(1)  # sleep 1 sec on waiting

    print("Successful disconnected with TWS")

Connecting...
Successfully connected


In [5]:
if ib.isConnected():
    try:  
        run_algorithm()
    except KeyboardInterrupt:
        disconnect()
        print('== Stop working ==')
else:
    print('Error: Not connected')

DateTime:  11-12-2019 at 12:14PM US/Eastern
Regular session
	Update SPY price history: SPY.csv
== Start working ==
	Update SPY price history: SPY.csv
	Check SMA for SPY (last=314.12): Short=314.0939 Long=314.0537
	Check SMA for SPY (last=314.12): Short=314.0917 Long=314.0470 (previous)
	Update SPY price history: SPY.csv
	Check SMA for SPY (last=314.15): Short=314.0906 Long=314.0332
	Check SMA for SPY (last=314.15): Short=314.0978 Long=314.0201 (previous)
	Update SPY price history: SPY.csv
	Check SMA for SPY (last=314.11): Short=314.0978 Long=314.0201
	Check SMA for SPY (last=314.11): Short=314.1067 Long=314.0057 (previous)
Successful disconnected with TWS
== Stop working ==


In [6]:
disconnect()

Successful disconnected with TWS
